In [1]:
import json
from typing import cast
from IPython.display import display, Markdown

from icecream import ic
import spacy
import textgraphs
import spacy_entity_linker as sel
import spacy_dbpedia_spotlight

/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch

/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages


In [2]:
with open("../data/wiki_guardians.json", "r") as fh:
    text: str = json.load(fh)["text"]
    paragraph = text.split("\n\n\n")[0]

In [3]:
paragraph

"Guardians of the Galaxy Vol. 3 (stylized in marketing as Guardians of the Galaxy Volume 3) is a 2023 American superhero film based on the Marvel Comics superhero team Guardians of the Galaxy, produced by Marvel Studios, and distributed by Walt Disney Studios Motion Pictures. It is the sequel to Guardians of the Galaxy (2014) and Guardians of the Galaxy Vol. 2 (2017), and the 32nd film in the Marvel Cinematic Universe (MCU). Written and directed by James Gunn, it features an ensemble cast including Chris Pratt, Zoe Saldaña, Dave Bautista, Karen Gillan, Pom Klementieff, Vin Diesel, Bradley Cooper, Will Poulter, Sean Gunn, Chukwudi Iwuji, Linda Cardellini, Nathan Fillion, and Sylvester Stallone. In the film, the Guardians must save Rocket's (Cooper) life from the High Evolutionary (Iwuji).\nGunn stated in November 2014 that he had initial ideas for a third and final film in the series, and announced his return to write and direct in April 2017. Disney fired him from the film in July 2018

## NER basic

In [15]:
nlp_ner_base = spacy.load("en_core_web_sm")
doc = nlp_ner_base(paragraph)
spacy.displacy.render(
    doc,
    style = "ent",
    jupyter = True,
)

## NER step with SpanMarker

In [8]:
nlp_ner = spacy.load("en_core_web_sm", exclude=["ner"])
nlp_ner.add_pipe("span_marker", config={"model": "tomaarsen/span-marker-mbert-base-multinerd"})

In [9]:
doc = nlp_ner(paragraph)

In [10]:
spacy.displacy.render(
    doc,
    style = "ent",
    jupyter = True,
)

## Entity Linking step

In [11]:
nlp_el = spacy.load("en_core_web_sm", exclude=["ner"])
nlp_el.add_pipe("span_marker", config={"model": "tomaarsen/span-marker-mbert-base-multinerd"})
nlp_el.add_pipe(
    'dbpedia_spotlight',
    config={
        'dbpedia_rest_endpoint': 'http://localhost:2222/rest', # for local docker DBpedia
        # 'dbpedia_rest_endpoint': 'https://api.dbpedia-spotlight.org/en', # for default with spacy-dbpedia-spotlight
        'overwrite_ents': True  # by default is set to True
    }  
)

In [12]:
doc = nlp_el(paragraph)

In [13]:
spacy.displacy.render(
    doc,
    style = "ent",
    jupyter = True,
)

In [14]:
for ent in doc.ents:
    print(
        ent.text,
        ent.label_,
        ent.kb_id_, 
        ent._.dbpedia_raw_result['@similarityScore']
    )

superhero film DBPEDIA_ENT http://dbpedia.org/resource/Superhero_film 1.0
Marvel Comics DBPEDIA_ENT http://dbpedia.org/resource/Marvel_Comics 0.999996715991464
superhero DBPEDIA_ENT http://dbpedia.org/resource/Superhero_film 1.0
Marvel Studios DBPEDIA_ENT http://dbpedia.org/resource/Marvel_Studios 1.0
Walt Disney Studios Motion Pictures DBPEDIA_ENT http://dbpedia.org/resource/Walt_Disney_Studios_Motion_Pictures 1.0
Marvel Cinematic Universe DBPEDIA_ENT http://dbpedia.org/resource/Marvel_Cinematic_Universe 1.0
James Gunn DBPEDIA_ENT http://dbpedia.org/resource/James_Gunn 1.0
ensemble cast DBPEDIA_ENT http://dbpedia.org/resource/Ensemble_cast 1.0
Chris Pratt DBPEDIA_ENT http://dbpedia.org/resource/Chris_Pratt 1.0
Zoe Saldaña DBPEDIA_ENT http://dbpedia.org/resource/Zoe_Saldaña 0.9999999882111298
Dave Bautista DBPEDIA_ENT http://dbpedia.org/resource/Dave_Bautista 1.0
Gillan DBPEDIA_ENT http://dbpedia.org/resource/Karen_Gillan 1.0
Pom Klementieff DBPEDIA_ENT http://dbpedia.org/resource/Pom_

In [16]:
ent._.dbpedia_raw_result

{'@URI': 'http://dbpedia.org/resource/Academy_Award_for_Best_Visual_Effects',
 '@support': '961',
 '@types': 'Wikidata:Q618779,DBpedia:Award',
 '@surfaceForm': 'Best Visual Effects',
 '@offset': '1867',
 '@similarityScore': '1.0',
 '@percentageOfSecondRank': '4.3046640449255563E-19'}

TODO
- parse ent._.dbpedia_raw_result -> into a pandas dataframe
- keep only 2 columns: similarityScore and types
- "push to Argilla"
- "explore"
  - use case 1 : order by similarityScore descending and take top 5 -> look at the least confident predictions
  - use case 2 : select Spans where types = 'Wikidata:Q618779,DBpedia:Award'

Dataset
- channel_1 = Spanmarker NER model
- channel_2 = DBPedia Spotlight Entity Linking
- channels = array of channels

Doc Task
- channel_1 = python generator entities of SpanMarker doc._.original_ents
- channel_2 = python generator entities of DBpedia = doc.ents

In [18]:
nlp_el.pipeline

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x2fe2ca5d0>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x2fe2cbc50>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x2b3f3f610>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x2b417c7d0>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x2b41bb2d0>),
 ('span_marker',
  <span_marker.spacy_integration.SpacySpanMarkerWrapper at 0x323e66190>),
 ('dbpedia_spotlight',
  <spacy_dbpedia_spotlight.entity_linker.EntityLinker at 0x2b4166f50>)]